In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from complexcgr import CGR
from pathlib import Path
from sklearn.model_selection import train_test_split
import dask
import dask.dataframe as dd
from dask import delayed
import seaborn as sns
import matplotlib.pyplot as plt

In [39]:
# Process the test data
def label_encoding_dim_category_reduction(df_in,country):
    # Print the processing status with the DataFrame name
    print("Processing DataFrame:", country)
    # Create a custom label encoding map
    custom_encoding_map = {'N': 0, 'A': 1, 'T': 2, 'G': 3, 'C': 4}
#     dropped_columns = ['Isolation source', 'category', 'BioSample', 'sra_accession', 'country']
#     snp_data = df_in.drop(columns=dropped_columns)
    # Apply custom label encoding to categorical columns
    categorical_cols = df_in.select_dtypes(include=['object']).columns
    df_in[categorical_cols] = df_in[categorical_cols].apply(lambda col: col.map(custom_encoding_map))

    # Combine non-categorical variables with informative SNP columns
#     data_combined = pd.concat([category_data[dropped_columns], df_filtered], axis=1)
    return df_in


In [68]:
# Function to process test data and make columns to be common betwrrn test and training data
def process_test_data(input_dataFrame, country):
    # metadata columns
    dropped_columns = ['Isolation source', 'category', 'BioSample', 'sra_accession', 'country']
    y_uganda=input_dataFrame[dropped_columns]
    # Drop unnecessary columns
    training_columns = X.columns.tolist()  # Assuming X is your training data

    # Check for missing columns in test data and add them with zeros
    missing_columns = set(training_columns) - set(input_dataFrame.columns)
    for column in missing_columns:
        input_dataFrame[column] = 0

    # Reorder the columns in the test data to match the training data
    test_data = input_dataFrame[training_columns]

    # Fill missing values (NaN) with zeros
    test_data = test_data.fillna(0)

    # Assuming there's a function called label_encoding_dim_category_reduction
    test_data = label_encoding_dim_category_reduction(test_data, country)
    # Add metadata
    test_data=pd.concat([y_uganda,test_data],axis=1)
    test_data=test_data.drop(columns=["Unnamed: 0"])

    return test_data



In [26]:
# Training data to extract columns
df_chisquare=pd.read_csv("chisquare_training_features_final.csv")
df_chisquare

,Unnamed: 0,Isolation source,category,BioSample,sra_accession,country,2027584,305245,4027906,3906518,...,3726424,2181910,4003673,2570482,3863338,3549846,297443,2099856,1852278,842668
0,49,feces swine,swine,SAMN03576991,SRR2847938,kenya,4.0,4.0,4.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0
1,142,chicken carcass,poultry,SAMN03577271,SRR3115981,ethipia,0.0,4.0,4.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,46,feces swine,swine,SAMN03577149,SRR3134909,ethipia,4.0,4.0,4.0,0.0,...,0.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0
3,16,pig stool,swine,SAMN07135217,SRR6951172,nigeria,4.0,4.0,4.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0
4,31,poultry,poultry,SAMN06240034,SRR5232014,uganda,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,148,feces poultry,poultry,SAMN03577373,SRR6411078,ethipia,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
708,44,chicken,poultry,SAMN02368606,SRR1106456,nigeria,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
709,7,pig stool,swine,SAMN07135265,SRR6950720,nigeria,4.0,4.0,4.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0
710,77,feces swine,swine,SAMN03577204,SRR3088941,ethipia,4.0,4.0,4.0,0.0,...,0.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0


In [30]:
# df.to_csv("updated_merged_training_data.csv")
X=df_chisquare.drop(columns=["category","BioSample","sra_accession","country","Isolation source"])
X

,Unnamed: 0,2027584,305245,4027906,3906518,4552348,1804684,620867,4719149,341365,...,3726424,2181910,4003673,2570482,3863338,3549846,297443,2099856,1852278,842668
0,49,4.0,4.0,4.0,0.0,4.0,0.0,4.0,4.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0
1,142,0.0,4.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,46,4.0,4.0,4.0,0.0,4.0,0.0,4.0,4.0,0.0,...,0.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0
3,16,4.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0
4,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
708,44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
709,7,4.0,4.0,4.0,0.0,4.0,0.0,4.0,4.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0
710,77,4.0,4.0,4.0,0.0,4.0,0.0,4.0,4.0,0.0,...,0.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0


In [45]:
#Test data
df_uganda=pd.read_csv("uganda_test_data.csv",low_memory=False)
df_tanzania=pd.read_csv("tanzania_test_data.csv",low_memory=False)
df_ethiopia=pd.read_csv("ethiopia_test_data.csv",low_memory=False)
df_kenya=pd.read_csv("kenya_test_data.csv",low_memory=False)
df_south_africa=pd.read_csv("south_africa_test_data.csv",low_memory=False)
df_tunisia=pd.read_csv("tunisia_test_data.csv",low_memory=False)
df_senegal=pd.read_csv("senegal_test_data.csv",low_memory=False)
df_mauritias=pd.read_csv("mauritias_test_data.csv",low_memory=False)
df_nigeria_bovine=pd.read_csv("nigeria_bovine_test_data.csv",low_memory=False)
df_nigeria_swine=pd.read_csv("nigeria_swine_test_data.csv",low_memory=False)
df_nigeria_poultry=pd.read_csv("nigeria_poultry_test_data.csv",low_memory=False)
df_sa_humna=pd.read_csv("south_africa_human_test_data.csv",low_memory=False)
df_tunisia_human=pd.read_csv("tunisia_human_test_data.csv",low_memory=False)

In [52]:
df_south_africa

,Unnamed: 0,Isolation source,BioSample,sra_accession,country,480,534,646,765,793,...,4562904,4562905,4562908,4562913,4562916,4562919,4585971,4607670,4815481,category
0,6,Poultry (Fecal),SAMN19025316,SRR14434437,south_africa,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,poultry
1,25,Bovine (Meat),SAMN19025259,SRR14434471,south_africa,G,C,T,T,C,...,N,N,N,N,N,N,N,N,N,bovine
2,14,Poultry (Fecal),SAMN19025305,SRR14434450,south_africa,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,poultry
3,10,Bovine (Meat),SAMN19025312,SRR14434442,south_africa,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,bovine
4,0,cow,SAMN11636367,SRR9050339,south_africa,G,C,T,T,C,...,N,N,N,N,N,N,N,N,N,bovine
5,3,Bovine (Meat),SAMN19025264,SRR14434433,south_africa,G,C,T,T,C,...,N,N,N,N,N,N,N,N,N,bovine
6,7,Bovine (Organ),SAMN19025262,SRR14434438,south_africa,G,C,T,T,C,...,N,N,N,N,N,N,N,N,N,bovine
7,42,Bovine (Fecal),SAMN19025257,SRR14434493,south_africa,G,C,T,T,C,...,N,N,N,N,N,N,N,N,N,bovine
8,32,Poultry (Meat),SAMN19025278,SRR14434479,south_africa,G,C,T,N,C,...,N,N,N,N,N,N,N,N,N,poultry
9,17,Bovine (Meat),SAMN19025299,SRR14434456,south_africa,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,bovine


In [69]:
# Process the test dataframes
ug_test=process_test_data(df_uganda,"uganda")
tz_test=process_test_data(df_tanzania,"tanzania")
eth_test=process_test_data(df_ethiopia,"ethipia")
keny_test=process_test_data(df_kenya,"kenya")
south_af_test=process_test_data(df_south_africa,"south_africa")
tuni_test=process_test_data(df_tunisia,"tunisia")
senegal_test=process_test_data(df_senegal,"senegal")
mau_test=process_test_data(df_mauritias,"mauritias")
nig_bo_test=process_test_data(df_nigeria_bovine,"nigeria_bovine")
nig_switest=process_test_data(df_nigeria_swine,"nigeria_swine")
nig_pol_test=process_test_data(df_nigeria_poultry,"nigeria_poultry")
south_africa_human_test=process_test_data(df_sa_humna,"south_africa_human")
tunisia_human_test=process_test_data(df_tunisia_human,"tuninisia_human")

Processing DataFrame: uganda
Processing DataFrame: tanzania
Processing DataFrame: ethipia
Processing DataFrame: kenya
Processing DataFrame: south_africa
Processing DataFrame: tunisia
Processing DataFrame: senegal
Processing DataFrame: mauritias
Processing DataFrame: nigeria_bovine
Processing DataFrame: nigeria_swine
Processing DataFrame: nigeria_poultry
Processing DataFrame: south_africa_human
Processing DataFrame: tuninisia_human


In [70]:
tunisia_human_test

,Isolation source,category,BioSample,sra_accession,country,2027584,305245,4027906,3906518,4552348,...,3726424,2181910,4003673,2570482,3863338,3549846,297443,2099856,1852278,842668
0,human,human,SAMN33426614,SRR23603253,Tunisia,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
1,human,human,SAMN33426600,SRR23603268,Tunisia,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
2,human,human,SAMN33426605,SRR23603263,Tunisia,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,human,human,SAMN33426569,SRR23603317,Tunisia,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,human,human,SAMN33426583,SRR23603287,Tunisia,4,4,4,4,4,...,0,4,4,0,4,0,4,4,0,4
5,human,human,SAMN33426596,SRR23603273,Tunisia,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
6,human,human,SAMN33426587,SRR23603283,Tunisia,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
7,human,human,SAMN33426612,SRR23603255,Tunisia,0,4,4,4,4,...,4,0,4,0,4,0,4,4,4,0
8,human,human,SAMN33426613,SRR23603254,Tunisia,0,4,4,0,4,...,4,4,0,4,0,0,0,0,0,0
9,human,human,SAMN33426599,SRR23603269,Tunisia,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0


In [71]:
# Concatenate all dfs
# Create a list of all the test dataframes
test_dfs = [ug_test, tz_test, eth_test, keny_test, south_af_test, tuni_test, senegal_test, mau_test, nig_bo_test, nig_switest, nig_pol_test, south_africa_human_test, tunisia_human_test]
# Concatenate the test dataframes into one
merged_test_df = pd.concat(test_dfs, ignore_index=True)
merged_test_df

,Isolation source,category,BioSample,sra_accession,country,2027584,305245,4027906,3906518,4552348,...,3726424,2181910,4003673,2570482,3863338,3549846,297443,2099856,1852278,842668
0,poultry,poultry,SAMN06240015,SRR5251507,uganda,0,4,0,0,4,...,0,4,0,0,0,0,4,0,0,4
1,poultry,poultry,SAMN06240023,SRR5232008,uganda,4,4,4,4,4,...,0,4,4,0,4,0,4,4,0,4
2,poultry,poultry,SAMN06238270,SRR5280386,uganda,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
3,poultry,poultry,SAMN06240082,SRR5278815,uganda,0,0,0,4,0,...,4,0,4,4,4,4,4,4,4,4
4,poultry,poultry,SAMN06240024,SRR5232016,uganda,4,4,4,4,4,...,0,4,4,0,4,0,4,4,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,human,human,SAMN33426609,SRR23603258,Tunisia,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
286,human,human,SAMN33426597,SRR23603272,Tunisia,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
287,human,human,SAMN33426593,SRR23603276,Tunisia,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
288,human,human,SAMN33426603,SRR23603265,Tunisia,0,0,0,4,0,...,4,0,4,4,4,4,4,4,4,4


In [72]:
merged_test_df.to_csv("test_data_all_chisquare.csv",index=False)

In [ ]:
# Test data
df_uganda=pd.read_csv("uganda_test_data.csv",low_memory=False)
# df_tz=pd.read_csv("tanzania_test_data.csv")
# df_test=pd.concat(df_ugands,df_tz)
df_uganda